# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jtr.readers as readers

[nltk_data] Downloading package punkt to /home/dirk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
transe_reader, cbilstm_snli_reader, distmult_reader, esim_snli_reader, cbilstm_snli_streaming_reader, cbow_xqa_reader, dam_snli_reader, modelf_reader, fastqa_reader, example_reader, complex_reader


### Create a reader

In [4]:
%%script bash
bash data/GloVe/download.sh

glove.6B.50d.txt already exists! Doing nothing!


In [5]:
from jtr.io.embeddings.embeddings import load_embeddings
from jtr.util.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"batch_size": 1, "repr_dim": 10, "repr_dim_input": embeddings.lookup.shape[1]}

In [6]:
# create example reader
from jtr.core import SharedResources

svac = SharedResources(vocab, config)
fastqa_reader = readers.readers["fastqa_reader"](svac)

### Setting up a reader from training data 

In [7]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [8]:
fastqa_reader.store("/tmp/fastqa_reader")

In [9]:
%%sh
ls /tmp/fastqa_reader/

checkpoint
model_module.data-00000-of-00001
model_module.index
model_module.meta
shared_resources


### Loading the reader

In [10]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [11]:
# or setup a new reader from file 
import tensorflow as tf
from jtr.util.vocab import Vocab

# reset graph -> computation graph is gone
tf.reset_default_graph()
svac = SharedResources(None, None)
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.setup_from_file("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


### Applying the reader

In [12]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in train_data]

In [13]:
for a in fastqa_reader(questions):
    print("{}, {}, {}".format(a.score, a.text, a.span))

0.00037656372296623886, the main drive (and in a, (565, 589)
0.00024976138956844807, grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a, (445, 589)
0.0005710708210244775, connects through 3 statues, (607, 633)
0.00024260827922262251, grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a, (445, 589)
0.00024483827291987836, grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a, (445, 589)
4.880869892076589e-05, station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year, (172, 453)
4.305880429456010